In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
from tensorflow import keras 
from tensorflow.keras import layers
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint



dataframe = pd.read_excel('triage dataset-coded 2.xlsx')

In [2]:
print(dataframe)

     gender   age  job  respcontact  placevisit  healthworker  fever  cough  \
0         0  30.0    1            1           1             1      1      0   
1         1  58.0    1            1           1             1      0      1   
2         0  41.0    1            0           0             1      0      0   
3         1  24.0    3            1           0             1      0      1   
4         0  48.0    1            1           1             1      0      1   
..      ...   ...  ...          ...         ...           ...    ...    ...   
435       1  60.0    6            1           0             0      1      1   
436       1  85.0    6            1           1             0      1      1   
437       1  55.0    6            1           1             0      1      1   
438       1  24.0    6            1           1             0      1      1   
439       0  29.0    1            0           1             1      0      0   

     sorethroat  vomdiarrhea  ...   ast    alt  ure

In [3]:
dataframe.shape

(440, 55)

In [4]:
dataframe = dataframe.dropna()

In [5]:
dataframe.head()

,gender,age,job,respcontact,placevisit,healthworker,fever,cough,sorethroat,vomdiarrhea,...,ast,alt,urea,creat,ldh,ck,crp,ferritin,inr,pcr
0,0,30.0,1,1,1,1,1,0,1,0,...,31.0,21.0,24.0,0.90,260.0,77.0,2.9,84.96,0.90,1
1,1,58.0,1,1,1,1,0,1,1,0,...,42.0,71.0,45.0,1.34,333.0,61.0,20.7,539.50,1.03,0
2,0,41.0,1,0,0,1,0,0,1,0,...,90.0,128.0,30.0,0.79,327.0,47.0,11.9,23.28,0.96,1
3,1,24.0,3,1,0,1,0,1,1,0,...,26.0,23.0,18.0,0.88,265.0,111.0,18.6,150.10,1.09,0
4,0,48.0,1,1,1,1,0,1,0,0,...,22.0,20.0,27.0,1.01,330.0,273.0,7.5,17.85,1.03,0


In [6]:
dataframe.shape

(293, 55)

In [7]:
val_dataframe = dataframe.sample(frac=0.2, random_state=1337)
train_dataframe = dataframe.drop(val_dataframe.index)

print(
    "Using %d samples for training and %d for validation"
    % (len(train_dataframe), len(val_dataframe))
)

Using 234 samples for training and 59 for validation


In [8]:
def dataframe_to_dataset(dataframe):
    dataframe = dataframe.copy()
    labels = dataframe.pop("pcr")
    ds = tf.data.Dataset.from_tensor_slices((dict(dataframe), labels))
    ds = ds.shuffle(buffer_size=len(dataframe))
    return ds


train_ds = dataframe_to_dataset(train_dataframe)
val_ds = dataframe_to_dataset(val_dataframe)

In [9]:
for x, y in train_ds.take(1):
    print("Input:", x)
    print("Target:", y)

Input: {'gender': <tf.Tensor: shape=(), dtype=int64, numpy=1>, 'age': <tf.Tensor: shape=(), dtype=float64, numpy=39.0>, 'job': <tf.Tensor: shape=(), dtype=int64, numpy=1>, 'respcontact': <tf.Tensor: shape=(), dtype=int64, numpy=0>, 'placevisit': <tf.Tensor: shape=(), dtype=int64, numpy=0>, 'healthworker': <tf.Tensor: shape=(), dtype=int64, numpy=1>, 'fever': <tf.Tensor: shape=(), dtype=int64, numpy=0>, 'cough': <tf.Tensor: shape=(), dtype=int64, numpy=1>, 'sorethroat': <tf.Tensor: shape=(), dtype=int64, numpy=0>, 'vomdiarrhea': <tf.Tensor: shape=(), dtype=int64, numpy=0>, 'malaise': <tf.Tensor: shape=(), dtype=int64, numpy=1>, 'smelltaste': <tf.Tensor: shape=(), dtype=int64, numpy=0>, 'smoking': <tf.Tensor: shape=(), dtype=int64, numpy=0>, 'pregnancy': <tf.Tensor: shape=(), dtype=int64, numpy=0>, 'immundiff': <tf.Tensor: shape=(), dtype=int64, numpy=0>, 'comorbid': <tf.Tensor: shape=(), dtype=int64, numpy=1>, 'dm': <tf.Tensor: shape=(), dtype=int64, numpy=0>, 'htn': <tf.Tensor: shape=(

In [10]:
train_ds = train_ds.batch(32)
val_ds = val_ds.batch(32)

In [11]:
from tensorflow.keras.layers.experimental.preprocessing import Normalization
from tensorflow.keras.layers.experimental.preprocessing import CategoryEncoding
from tensorflow.keras.layers.experimental.preprocessing import StringLookup


def encode_numerical_feature(feature, name, dataset):
    # Create a Normalization layer for our feature
    normalizer = Normalization()

    # Prepare a Dataset that only yields our feature
    feature_ds = dataset.map(lambda x, y: x[name])
    feature_ds = feature_ds.map(lambda x: tf.expand_dims(x, -1))

    # Learn the statistics of the data
    normalizer.adapt(feature_ds)

    # Normalize the input feature
    encoded_feature = normalizer(feature)
    return encoded_feature


def encode_string_categorical_feature(feature, name, dataset):
    # Create a StringLookup layer which will turn strings into integer indices
    index = StringLookup()

    # Prepare a Dataset that only yields our feature
    feature_ds = dataset.map(lambda x, y: x[name])
    feature_ds = feature_ds.map(lambda x: tf.expand_dims(x, -1))

    # Learn the set of possible string values and assign them a fixed integer index
    index.adapt(feature_ds)

    # Turn the string input into integer indices
    encoded_feature = index(feature)

    # Create a CategoryEncoding for our integer indices
    encoder = CategoryEncoding(output_mode="binary")

    # Prepare a dataset of indices
    feature_ds = feature_ds.map(index)

    # Learn the space of possible indices
    encoder.adapt(feature_ds)

    # Apply one-hot encoding to our indices
    encoded_feature = encoder(encoded_feature)
    return encoded_feature


def encode_integer_categorical_feature(feature, name, dataset):
    # Create a CategoryEncoding for our integer indices
    encoder = CategoryEncoding(output_mode="binary")

    # Prepare a Dataset that only yields our feature
    feature_ds = dataset.map(lambda x, y: x[name])
    feature_ds = feature_ds.map(lambda x: tf.expand_dims(x, -1))

    # Learn the space of possible indices
    encoder.adapt(feature_ds)

    # Apply one-hot encoding to our indices
    encoded_feature = encoder(feature)
    return encoded_feature

In [12]:
# Categorical features encoded as integers
gender= keras.Input(shape=(1,), name="gender", dtype="int64")
job= keras.Input(shape=(1,), name="job", dtype="int64")
respcontact= keras.Input(shape=(1,), name="respcontact", dtype="int64")
placevisit= keras.Input(shape=(1,), name="placevisit", dtype="int64")
healthworker= keras.Input(shape=(1,), name="healthworker", dtype="int64")
fever= keras.Input(shape=(1,), name="fever", dtype="int64")
cough= keras.Input(shape=(1,), name="cough", dtype="int64")
sorethroat= keras.Input(shape=(1,), name="sorethroat", dtype="int64")
vomdiarrhea= keras.Input(shape=(1,), name="vomdiarrhea", dtype="int64")
malaise= keras.Input(shape=(1,), name="malaise", dtype="int64")
smelltaste= keras.Input(shape=(1,), name="smelltaste", dtype="int64")
smoking= keras.Input(shape=(1,), name="smoking", dtype="int64") 
pregnancy= keras.Input(shape=(1,), name="pregnancy", dtype="int64")
immundiff= keras.Input(shape=(1,), name="immundiff", dtype="int64")
comorbid= keras.Input(shape=(1,), name="comorbid", dtype="int64")
dm= keras.Input(shape=(1,), name="dm", dtype="int64")
htn= keras.Input(shape=(1,), name="htn", dtype="int64")
cardiac= keras.Input(shape=(1,), name="cardiac", dtype="int64")
ckd= keras.Input(shape=(1,), name="ckd", dtype="int64")
chest= keras.Input(shape=(1,), name="chest", dtype="int64")
hepatic= keras.Input(shape=(1,), name="hepatic", dtype="int64")
cancer= keras.Input(shape=(1,), name="cancer", dtype="int64")
others= keras.Input(shape=(1,), name="others", dtype="int64")

# Categorical feature encoded as string
# thal = keras.Input(shape=(1,), name="thal", dtype="string")

# Numerical features
age= keras.Input(shape=(1,), name="age")
wbc= keras.Input(shape=(1,), name="wbc")
neucount= keras.Input(shape=(1,), name="neucount")
neu= keras.Input(shape=(1,), name="neu")
lymcount= keras.Input(shape=(1,), name="lymcount")
lym= keras.Input(shape=(1,), name="lym")
monocount= keras.Input(shape=(1,), name="monocount")
mono= keras.Input(shape=(1,), name="mono")
eoscount= keras.Input(shape=(1,), name="eoscount")
eos= keras.Input(shape=(1,), name="eos")
basocount= keras.Input(shape=(1,), name="basocount")
baso= keras.Input(shape=(1,), name="baso")
rbc= keras.Input(shape=(1,), name="rbc")
hgb= keras.Input(shape=(1,), name="hgb")
hct= keras.Input(shape=(1,), name="hct")
mcv= keras.Input(shape=(1,), name="mcv")
mch= keras.Input(shape=(1,), name="mch")
mchc= keras.Input(shape=(1,), name="mchc")
rdw= keras.Input(shape=(1,), name="rdw")
plt= keras.Input(shape=(1,), name="plt")
mpv= keras.Input(shape=(1,), name="mpv")
neutolym= keras.Input(shape=(1,), name="neutolym")
ast= keras.Input(shape=(1,), name="ast")
alt= keras.Input(shape=(1,), name="alt")
urea= keras.Input(shape=(1,), name="urea")
creat= keras.Input(shape=(1,), name="creat")
ldh= keras.Input(shape=(1,), name="ldh")
ck= keras.Input(shape=(1,), name="ck")
crp= keras.Input(shape=(1,), name="crp")
ferritin= keras.Input(shape=(1,), name="ferritin")
inr= keras.Input(shape=(1,), name="inr")

all_inputs = [
    gender,
    age,
    job,
    respcontact,
    placevisit,
    healthworker,
    fever,
    cough,
    sorethroat,
    vomdiarrhea,
    malaise,
    smelltaste,
    smoking,
    pregnancy,
    immundiff,
    comorbid,
    dm,
    htn,
    cardiac,
    ckd,
    chest,
    hepatic,
    cancer,
    others,
    wbc,
    neucount,
    neu,
    lymcount,
    lym,
    monocount,
    mono,
    eoscount,
    eos,
    basocount,
    baso,
    rbc,
    hgb,
    hct,
    mcv,
    mch,
    mchc,
    rdw,
    plt,
    mpv,
    neutolym,
    ast,
    alt,
    urea,
    creat,
    ldh,
    ck,
    crp,
    ferritin,
    inr,
]

# Integer categorical features
gender_encoded = encode_integer_categorical_feature(gender, "gender", train_ds)
job_encoded = encode_integer_categorical_feature(job, "job", train_ds)
respcontact_encoded = encode_integer_categorical_feature(respcontact, "respcontact", train_ds)
placevisit_encoded = encode_integer_categorical_feature(placevisit, "placevisit", train_ds)
healthworker_encoded = encode_integer_categorical_feature(healthworker, "healthworker", train_ds)
fever_encoded = encode_integer_categorical_feature(fever, "fever", train_ds)
cough_encoded = encode_integer_categorical_feature(cough, "cough", train_ds)
sorethroat_encoded = encode_integer_categorical_feature(sorethroat, "sorethroat", train_ds)
vomdiarrhea_encoded = encode_integer_categorical_feature(vomdiarrhea, "vomdiarrhea", train_ds)
malaise_encoded = encode_integer_categorical_feature(malaise, "malaise", train_ds)
smelltaste_encoded = encode_integer_categorical_feature(smelltaste, "smelltaste", train_ds)
smoking_encoded = encode_integer_categorical_feature(smoking, "smoking", train_ds)
pregnancy_encoded = encode_integer_categorical_feature(pregnancy, "pregnancy", train_ds)
immundiff_encoded = encode_integer_categorical_feature(immundiff, "immundiff", train_ds)
comorbid_encoded = encode_integer_categorical_feature(comorbid, "comorbid", train_ds)
dm_encoded = encode_integer_categorical_feature(dm, "dm", train_ds)
htn_encoded = encode_integer_categorical_feature(htn, "htn", train_ds)
cardiac_encoded = encode_integer_categorical_feature(cardiac, "cardiac", train_ds)
ckd_encoded = encode_integer_categorical_feature(ckd, "ckd", train_ds)
chest_encoded = encode_integer_categorical_feature(chest, "chest", train_ds)
hepatic_encoded = encode_integer_categorical_feature(hepatic, "hepatic", train_ds)
cancer_encoded = encode_integer_categorical_feature(cancer, "cancer", train_ds)
others_encoded = encode_integer_categorical_feature(others, "others", train_ds)

# String categorical features
# thal_encoded = encode_string_categorical_feature(thal, "thal", train_ds)

# Numerical features
age_encoded = encode_numerical_feature(age, "age", train_ds)
wbc_encoded = encode_numerical_feature(wbc, "wbc", train_ds)
neucount_encoded = encode_numerical_feature(neucount, "neucount", train_ds)
neu_encoded = encode_numerical_feature(neu, "neu", train_ds)
lymcount_encoded = encode_numerical_feature(lymcount, "lymcount", train_ds)
lym_encoded = encode_numerical_feature(lym, "lym", train_ds)
monocount_encoded = encode_numerical_feature(monocount, "monocount", train_ds)
mono_encoded = encode_numerical_feature(mono, "mono", train_ds)
eoscount_encoded = encode_numerical_feature(eoscount, "eoscount", train_ds)
eos_encoded = encode_numerical_feature(basocount, "basocount", train_ds)
basocount_encoded = encode_numerical_feature(age, "age", train_ds)
baso_encoded = encode_numerical_feature(basocount, "basocount", train_ds)
rbc_encoded = encode_numerical_feature(rbc, "rbc", train_ds)
hgb_encoded = encode_numerical_feature(hgb, "hgb", train_ds)
hct_encoded = encode_numerical_feature(hct, "hct", train_ds)
mcv_encoded = encode_numerical_feature(mcv, "mcv", train_ds)
mch_encoded = encode_numerical_feature(mch, "mch", train_ds)
mchc_encoded = encode_numerical_feature(mchc, "mchc", train_ds)
rdw_encoded = encode_numerical_feature(rdw, "rdw", train_ds)
plt_encoded = encode_numerical_feature(plt, "plt", train_ds)
mpv_encoded = encode_numerical_feature(mpv, "mpv", train_ds)
neutolym_encoded = encode_numerical_feature(neutolym, "neutolym", train_ds)
ast_encoded = encode_numerical_feature(ast, "ast", train_ds)
alt_encoded = encode_numerical_feature(alt, "alt", train_ds)
urea_encoded = encode_numerical_feature(urea, "urea", train_ds)
creat_encoded = encode_numerical_feature(creat, "creat", train_ds)
ldh_encoded = encode_numerical_feature(ldh, "ldh", train_ds)
ck_encoded = encode_numerical_feature(ck, "ck", train_ds)
crp_encoded = encode_numerical_feature(crp, "crp", train_ds)
ferritin_encoded = encode_numerical_feature(ferritin, "ferritin", train_ds)
inr_encoded = encode_numerical_feature(inr, "inr", train_ds)

all_features = layers.concatenate(
    [
        gender_encoded,
        age_encoded,
        job_encoded,
        respcontact_encoded,
        placevisit_encoded,
        healthworker_encoded,
        fever_encoded,
        cough_encoded,
        sorethroat_encoded,
        vomdiarrhea_encoded,
        malaise_encoded,
        smelltaste_encoded,
        smoking_encoded,
        pregnancy_encoded,
        immundiff_encoded,
        comorbid_encoded,
        dm_encoded,
        htn_encoded,
        cardiac_encoded,
        ckd_encoded,
        chest_encoded,
        hepatic_encoded,
        cancer_encoded,
        others_encoded,
        wbc_encoded,
        neucount_encoded,
        neu_encoded,
        lymcount_encoded,
        lym_encoded,
        monocount_encoded,
        mono_encoded,
        eoscount_encoded,
        eos_encoded,
        basocount_encoded,
        baso_encoded,
        rbc_encoded,
        hgb_encoded,
        hct_encoded,
        mcv_encoded,
        mch_encoded,
        mchc_encoded,
        rdw_encoded,
        plt_encoded,
        mpv_encoded,
        neutolym_encoded,
        ast_encoded,
        alt_encoded,
        urea_encoded,
        creat_encoded,
        ldh_encoded,
        ck_encoded,
        crp_encoded,
        ferritin_encoded,
        inr_encoded,
    ]
)

# initialize the initial learning rate, number of epochs to train for,
# and batch size
INIT_LR = 1e-4
EPOCHS = 20
BS = 32


x = keras.layers.Dense(32)(all_features)
x = layers.BatchNormalization()(x)
x = layers.LeakyReLU()(x)
x = keras.layers.Dropout(0.5)(x)
x = keras.layers.Dense(16)(x)
x = layers.BatchNormalization()(x)
x = layers.LeakyReLU()(x)
x = keras.layers.Dropout(0.5)(x)
x = keras.layers.Dense(8)(x)
x = layers.BatchNormalization()(x)
x = layers.LeakyReLU()(x)
x = keras.layers.Dropout(0.5)(x)
output = keras.layers.Dense(1, activation="sigmoid")(x)

model = keras.Model(all_inputs, output)

model.compile(loss="binary_crossentropy", optimizer="Adam",metrics=["accuracy"])

In [13]:
# `rankdir='LR'` is to make the graph horizontal.
keras.utils.plot_model(model, show_shapes=True, rankdir="LR")

('Failed to import pydot. You must `pip install pydot` and install graphviz (https://graphviz.gitlab.io/download/), ', 'for `pydotprint` to work.')


In [14]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
gender (InputLayer)             [(None, 1)]          0                                            
__________________________________________________________________________________________________
age (InputLayer)                [(None, 1)]          0                                            
__________________________________________________________________________________________________
job (InputLayer)                [(None, 1)]          0                                            
__________________________________________________________________________________________________
respcontact (InputLayer)        [(None, 1)]          0                                            
______________________________________________________________________________________________

Total params: 3,614
Trainable params: 3,409
Non-trainable params: 205
__________________________________________________________________________________________________


In [16]:
from tensorflow.keras.callbacks import TensorBoard

NAME = "triage-v3"
tensorboard = TensorBoard(log_dir="logs/{}".format(NAME))

In [17]:
# checkpoint
#filepath="weights.best.hdf5"
#checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
#callbacks_list = [checkpoint]

model.fit(train_ds, epochs=10, validation_data=val_ds, callbacks=[tensorboard])

Epoch 1/10
8/8 [==============================] - 7s 359ms/step - loss: 0.7787 - accuracy: 0.5538 - val_loss: 0.6732 - val_accuracy: 0.5424
Epoch 2/10
8/8 [==============================] - 0s 26ms/step - loss: 0.7282 - accuracy: 0.4965 - val_loss: 0.6629 - val_accuracy: 0.5932
Epoch 3/10
8/8 [==============================] - 0s 38ms/step - loss: 0.8222 - accuracy: 0.5223 - val_loss: 0.6575 - val_accuracy: 0.5932
Epoch 4/10
8/8 [==============================] - 0s 29ms/step - loss: 0.7840 - accuracy: 0.5163 - val_loss: 0.6543 - val_accuracy: 0.6102
Epoch 5/10
8/8 [==============================] - 1s 83ms/step - loss: 0.7455 - accuracy: 0.5273 - val_loss: 0.6511 - val_accuracy: 0.5932
Epoch 6/10
8/8 [==============================] - 0s 45ms/step - loss: 0.8029 - accuracy: 0.4732 - val_loss: 0.6496 - val_accuracy: 0.6271
Epoch 7/10
8/8 [==============================] - 0s 32ms/step - loss: 0.7374 - accuracy: 0.5408 - val_loss: 0.6501 - val_accuracy: 0.6271
Epoch 8/10
8/8 [==========

In [16]:
# model.save('triage-v3-0.7119')
